In [1]:
import numpy as np

In [4]:
def fedavg(models:list):
    return np.mean(models)

In [10]:
def krum(models:list):
    return np.max(models)

In [45]:
def aggregator(func, *data):
    return func(data)

In [46]:
a = fedavg
aggregator(a, [1,2])

1.5

In [48]:
a = [1,2,3,]
a.remove(1)
a

[2, 3]

In [24]:
import random

In [38]:
random.randint(0,3)

2

In [63]:
from sklearn.cluster import KMeans
import numpy as np
a = {1:2, 2:4, 3:34, 4:3, 5:23}
X = np.array(list(a.values())).reshape(-1, 1)
k_means = KMeans(n_clusters=1)
k_means.fit(X)
k_means_cluster_centers = k_means.cluster_centers_
np.mean(k_means_cluster_centers)

/home/chao/git/beidou/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


15.75

In [73]:
mylist = ["apple", "banana", "cherry"]
mylist= random.shuffle(mylist)
mylist

In [100]:
from sklearn.mixture import BayesianGaussianMixture
from sklearn.mixture import GaussianMixture

In [171]:
a = {0: 0.050472747534513474, 1: 0.33852526545524597, 2: 0.3366793096065521, 3: 0.322217732667923, 4: 0.3181234896183014, 5: 0.34658750891685486, 6: 0.30072730779647827, 7: 0.31256982684135437, 8: 0.3395855724811554, 9: 1.0}
X = np.array(list(a.values()))

In [93]:
def fit_bayesian_gmm(data, max_components=3):
    bgmm = BayesianGaussianMixture(n_components=max_components, random_state=0)
    bgmm.fit(data.reshape(-1, 1))
    return bgmm

def get_effective_components(bgmm, threshold=1e-3):
    effective_components = np.sum(bgmm.weights_ > threshold)
    return effective_components

In [109]:
bgmm = GaussianMixture(n_components=3, random_state=0)
bgmm.fit_predict(X.reshape(-1, 1))

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [102]:
def select_gmm_components(data, max_components=5):
    aic = []
    bic = []
    models = []
    
    for n in range(1, max_components + 1):
        gmm = GaussianMixture(n_components=n, random_state=0)
        gmm.fit(data.reshape(-1, 1))
        aic.append(gmm.aic(data.reshape(-1, 1)))
        bic.append(gmm.bic(data.reshape(-1, 1)))
        models.append(gmm)
    
    return aic, bic, models

In [110]:
aic1, bic1, models1 = select_gmm_components(X)

In [149]:
b = a.values()
X = np.array(list(b))

In [150]:
from sklearn.cluster import MeanShift
mean_shift = MeanShift()
mean_shift.fit_predict(X.reshape(-1,1))

array([0, 0, 1, 1, 2, 4, 3, 0])

In [120]:
mean_shift.cluster_centers_

array([[0.326877  ],
       [1.        ],
       [0.05047275]])

In [111]:
aic1, bic1

([2.727203986385586,
  -13.684793638169815,
  -39.80753849865127,
  -38.64296804532699,
  -37.56682305383056],
 [3.3323741723736777,
  -12.171868173199584,
  -37.3868577546989,
  -35.314532022392484,
  -33.33063175191392])

In [74]:
mylist

In [151]:
from sklearn.cluster import OPTICS
opt = OPTICS()
opt.fit_predict(X.reshape(-1,1))

array([0, 0, 0, 0, 0, 0, 0, 0])

In [157]:
from sklearn.cluster import DBSCAN
db=DBSCAN(eps=0.1, min_samples=1)
clusters = db.fit_predict(X.reshape(-1,1))

In [158]:
cluster_points = X[clusters == 0]

In [173]:
db = DBSCAN(eps=0.1, min_samples=1)
clusters = db.fit_predict(X.reshape(-1,1))
unique_labels = set(clusters)

if len(unique_labels) > 1:
    trigger = True            

    lowerbound = []
    upperbound = []
    
    for label in unique_labels:
        cluster_points = X[clusters == label]
        lowerbound.append(min(cluster_points))
        upperbound.append(max(cluster_points))
    
    # the threshold is the average of lowerbound of the first class and upperbound of the second class
    rep_threshold = np.mean([lowerbound[-1], upperbound[-2]])

In [175]:
lowerbound

[0.050472747534513474, 0.30072730779647827, 1.0]

In [176]:
upperbound

[0.050472747534513474, 0.34658750891685486, 1.0]

In [174]:
rep_threshold

0.6732937544584274

In [167]:
c = list(b)
c.sort()
c[-2]

0.3395855724811554

In [159]:
cluster_points

array([0.33852527, 0.33667931, 0.32221773, 0.31812349, 0.34658751,
       0.30072731, 0.31256983, 0.33958557])

In [62]:
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers

array([[ 3. ],
       [28.5]])

In [148]:
a = {1: 0.33852526545524597, 2: 0.3366793096065521, 3: 0.322217732667923, 4: 0.3181234896183014, 5: 0.34658750891685486, 6: 0.30072730779647827, 7: 0.31256982684135437, 8: 0.3395855724811554}
a.values()

dict_values([0.33852526545524597, 0.3366793096065521, 0.322217732667923, 0.3181234896183014, 0.34658750891685486, 0.30072730779647827, 0.31256982684135437, 0.3395855724811554])

In [8]:
aggregator(fedavg, a)

13.666666666666666

In [11]:
aggregator(krum, a)

34

In [15]:
from typing import Union
import random

In [13]:
type(1)==int

True

In [21]:
from util import generate_attack_matrix
    

In [23]:
node_list = list(range(10))
attack_type = 'label flipping'
targeted = False
poisoned_node_ratio = 20
noise_injected_ratio = 20
poisoned_sample_ratio = 100
generate_attack_matrix(node_list, attack_type, targeted, poisoned_node_ratio, noise_injected_ratio, poisoned_sample_ratio)

attacked_node_list_[8, 0]


{0: {'attack_type': 'label flipping',
  'targeted': False,
  'noise_injected_ratio': 20,
  'poisoned_sample_ratio': 100},
 1: {'attack_type': 'no attack',
  'targeted': None,
  'noise_injected_ratio': None,
  'poisoned_sample_ratio': None},
 2: {'attack_type': 'no attack',
  'targeted': None,
  'noise_injected_ratio': None,
  'poisoned_sample_ratio': None},
 3: {'attack_type': 'no attack',
  'targeted': None,
  'noise_injected_ratio': None,
  'poisoned_sample_ratio': None},
 4: {'attack_type': 'no attack',
  'targeted': None,
  'noise_injected_ratio': None,
  'poisoned_sample_ratio': None},
 5: {'attack_type': 'no attack',
  'targeted': None,
  'noise_injected_ratio': None,
  'poisoned_sample_ratio': None},
 6: {'attack_type': 'no attack',
  'targeted': None,
  'noise_injected_ratio': None,
  'poisoned_sample_ratio': None},
 7: {'attack_type': 'no attack',
  'targeted': None,
  'noise_injected_ratio': None,
  'poisoned_sample_ratio': None},
 8: {'attack_type': 'label flipping',
  'targ

In [44]:
from typing import OrderedDict, List
import torch
import logging

In [ ]:
def cosine_metric2(model1: OrderedDict[str, torch.Tensor], model2: OrderedDict[str, torch.Tensor], similarity: bool = False) -> Optional[float]:
    if model1 is None or model2 is None:
        logging.info("Cosine similarity cannot be computed due to missing model")
        return None

    cos_similarities = []

    for layer in model1:
        if layer in model2:
            l1 = model1[layer].flatten()
            l2 = model2[layer].flatten()
            if l1.shape != l2.shape:
                # Adjust the shape of the smaller layer to match the larger layer
                min_len = min(l1.shape[0], l2.shape[0])
                l1, l2 = l1[:min_len], l2[:min_len]

            cos_sim = torch.nn.functional.cosine_similarity(l1.unsqueeze(0), l2.unsqueeze(0), dim=1)
            cos_similarities.append(cos_sim.item())

    if cos_similarities:
        avg_cos_sim = torch.mean(torch.tensor(cos_similarities))
        # result = torch.clamp(avg_cos_sim, min=0).item()
        # return result
        return avg_cos_sim.item() if similarity else (1 - avg_cos_sim.item())
    else:
        return None
    
def cosine_metric(model1: OrderedDict, model2: OrderedDict, similarity: bool = False) -> Optional[float]:
    if model1 is None or model2 is None:
        logging.info("Cosine similarity cannot be computed due to missing model")
        return None

    cos_similarities: List = []

    for layer in model1:
        if layer in model2:
            l1 = model1[layer].to('cpu')
            l2 = model2[layer].to('cpu')
            if l1.shape != l2.shape:
                # Adjust the shape of the smaller layer to match the larger layer
                min_len = min(l1.shape[0], l2.shape[0])
                l1, l2 = l1[:min_len], l2[:min_len]
            cos = torch.nn.CosineSimilarity(dim=l1.dim() - 1)
            cos_mean = torch.mean(cos(l1.float(), l2.float())).mean()
            cos_similarities.append(cos_mean)
        else:
            logging.info("Layer {} not found in model 2".format(layer))

    if cos_similarities:    
        cos = torch.Tensor(cos_similarities)
        avg_cos = torch.mean(cos)
        relu_cos = torch.nn.functional.relu(avg_cos)  # relu to avoid negative values
        return relu_cos.item() if similarity else (1 - relu_cos.item())
    else:
        return None
        

def euclidean_metric(model1: OrderedDict[str, torch.Tensor], model2: OrderedDict[str, torch.Tensor], standardized: bool = False, similarity: bool = False) -> Optional[float]:
    if model1 is None or model2 is None:
        return None

    distances = []

    for layer in model1:
        if layer in model2:
            l1 = model1[layer].flatten()
            l2 = model2[layer].flatten()
            if standardized:
                l1 = (l1 - l1.mean()) / l1.std()
                l2 = (l2 - l2.mean()) / l2.std()
            
            distance = torch.norm(l1 - l2, p=2)
            if similarity:
                norm_sum = torch.norm(l1, p=2) + torch.norm(l2, p=2)
                similarity_score = 1 - (distance / norm_sum if norm_sum != 0 else 0)
                distances.append(similarity_score.item())
            else:
                distances.append(distance.item())

    if distances:
        avg_distance = torch.mean(torch.tensor(distances))
        return avg_distance.item()
    else:
        return None
    

def minkowski_metric(model1: OrderedDict[str, torch.Tensor], model2: OrderedDict[str, torch.Tensor], p: int, similarity: bool = False) -> Optional[float]:
    if model1 is None or model2 is None:
        return None

    distances = []

    for layer in model1:
        if layer in model2:
            l1 = model1[layer].flatten()
            l2 = model2[layer].flatten()

            distance = torch.norm(l1 - l2, p=p)
            if similarity:
                norm_sum = torch.norm(l1, p=p) + torch.norm(l2, p=p)
                similarity_score = 1 - (distance / norm_sum if norm_sum != 0 else 0)
                distances.append(similarity_score.item())
            else:
                distances.append(distance.item())

    if distances:
        avg_distance = torch.mean(torch.tensor(distances))
        return avg_distance.item()
    else:
        return None

def chebyshev_metric(model1: OrderedDict[str, torch.Tensor], model2: OrderedDict[str, torch.Tensor], similarity: bool = False) -> Optional[float]:
    if model1 is None or model2 is None:
        return None

    distances = []

    for layer in model1:
        if layer in model2:
            l1 = model1[layer].flatten()
            l2 = model2[layer].flatten()

            distance = torch.norm(l1 - l2, p=float('inf'))
            if similarity:
                norm_sum = torch.norm(l1, p=float('inf')) + torch.norm(l2, p=float('inf'))
                similarity_score = 1 - (distance / norm_sum if norm_sum != 0 else 0)
                distances.append(similarity_score.item())
            else:
                distances.append(distance.item())

    if distances:
        avg_distance = torch.mean(torch.tensor(distances))
        return avg_distance.item()
    else:
        return None


def manhattan_metric(model1: OrderedDict[str, torch.Tensor], model2: OrderedDict[str, torch.Tensor], similarity: bool = False) -> Optional[float]:
    if model1 is None or model2 is None:
        return None

    distances = []

    for layer in model1:
        if layer in model2:
            l1 = model1[layer].flatten()
            l2 = model2[layer].flatten()

            distance = torch.norm(l1 - l2, p=1)
            if similarity:
                norm_sum = torch.norm(l1, p=1) + torch.norm(l2, p=1)
                similarity_score = 1 - (distance / norm_sum if norm_sum != 0 else 0)
                distances.append(similarity_score.item())
            else:
                distances.append(distance.item())

    if distances:
        avg_distance = torch.mean(torch.tensor(distances))
        return avg_distance.item()
    else:
        return None


def pearson_correlation_metric(model1: OrderedDict[str, torch.Tensor], model2: OrderedDict[str, torch.Tensor], similarity: bool = False) -> Optional[float]:
    if model1 is None or model2 is None:
        return None

    correlations = []

    for layer in model1:
        if layer in model2:
            l1 = model1[layer].flatten()
            l2 = model2[layer].flatten()

            if l1.shape != l2.shape:
                min_len = min(l1.shape[0], l2.shape[0])
                l1, l2 = l1[:min_len], l2[:min_len]

            correlation = torch.corrcoef(torch.stack((l1, l2)))[0, 1]
            if similarity:
                adjusted_similarity = (correlation + 1) / 2
                correlations.append(adjusted_similarity.item())
            else:
                correlations.append(1 - (correlation + 1) / 2)

    if correlations:
        avg_correlation = torch.mean(torch.tensor(correlations))
        return avg_correlation.item()
    else:
        return None

In [43]:
def get_repution(repution_func, model, current_round_nei_models):
    nei_repution_score = {}
    for nei in current_round_nei_models:
        nei_repution_score[nei] = repution_func(model, current_round_nei_models[nei])
    return nei_repution_score